Imports

viele von diese Bibliotheken muessen mit pip intalliert werden.

wenn einige nicht funktionnieren, kann die Installation im Terminal mit folgendes ausprobiert werden :

conda install 'bibliotheke'

python3 -m pip install 'bibliotheke'

In [13]:
import tkinter as tk
from PIL import Image, ImageTk 
from tkinter import filedialog, messagebox  #messagebox wird jetzt noch nicht benutzt 
from tkinter import PhotoImage
import pandas as pd
import PyPDF2                                  # 
import pytesseract
pytesseract.pytesseract.tesseract_cmd ="C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
from docx import Document   
import os
import openpyxl


Tkinter Hauptfenster

In [14]:
# Wir stellen den Root Fenster an
root = tk.Tk()                                                                  # Funktion die unser Hauptfenster erstellt
root.geometry("550x300")                                                        # wir definieren die groeße des Fensters
root.title("Speakeasy v0.1")                                                    # Wir definieren den Titel
root.resizable(width=True, height=True)                                         # Wir setzten eine dynamische Groeße ein : wenn wir ein großes Bild z.B zufuegen, wir das Fenster automatisch angepasst.

# Wir laden unser Speakeasy Logo !
logo_path = r"C:\Users\Admin\Desktop\Speakeasy\data\logo_Speakeasy_1080p.png"       # right clic, "copy Path", als string eingeben, tada.
logo = Image.open(logo_path)                                                        # Wir benutzen die Bibliothek PIL um den Bild zu oeffnen
logo = logo.resize((200, 200))                                                      # Wir skalieren das importierte bild mit .resize((length, width)) doppel Klammer !
global_logo = ImageTk.PhotoImage(logo)                                              # Wir konvertieren unser durch Pillow (PIL) geaendertes Bild in einen Format der von Tkinter uebernommen werden kann mit PhotoImage() 


# Herstekkung des Label, um das Bild anzeigen zu koennen.
label = tk.Label(root,                                                                                              # Wir setzten den Laben auf unser Hauptfenster
                 text='Willkommen bei Speakeasy.\nBitte wählen Sie eine Datei aus, um den Text zu extrahieren',     # Wir sind gut erzogen, sagen Hallo und wie wir behelflich sein koennen.
                 image=global_logo, 
                 compound='top')
label.pack(anchor=tk.CENTER)                                                                                        # Wir Positionnieren unser Label in der Mitte



Funktionen, die die verschiedene Dataien oeffnen:

"Wir stellen Schlusseln her, die Tueren offnen konnen"

In [15]:
# Funktion, um Excel-Dateien zu lesen (https://statisquo.de/2019/01/14/excel-dateien-in-python-importieren-mit-pandas/)

def read_excel(file_path):
    try:
        df = pd.read_excel(file_path)                       # .read_excel liest Excel-Dateien
        return df.to_string()                               # Wandelt Inhalt in String um
    except Exception as e:                                  # 'Exception as e' fängt Fehler ab. e = Variable für Fehler 
        print(f'Fehler beim Lesen der Excel-Datei: {e}')    # Fehlermeldung (Später als Messagebox)
        return None                                         # Bei Fehler Ende der Funktion




In [16]:
# Funktion, um Word-Dokumente zu lesen (https://python-docx.readthedocs.io/en/latest/)

def read_word(file_path):
    try:
        document = Document(file_path)                      # 'Document' ist eine Klasse aus der Library um Word-Dokumente zu laden (neben den Imports schon beschrieben, aber safety first)
        text = []                                           # Leere Liste die den Text später speichert, um die Textstruktur beizubehalten. Also die Absätze zum Beispiel
        for paragraph in document.paragraphs:               # Geht alle Absätze durch und sorgt dafür, dass jeder Absatz als 'paragraph' gespeichert wird
            text.append(paragraph.text)                     # Jeder Textinhalt eines Absatzes wird in die Liste 'text' eingefügt 
        return "\n".join(text)                              # Gibt den kompletten Text als String zurück und Zeilenumbruch für Absätze 
    except Exception as e:
        print(f'Fehler beim Lesen des Word-Dokuments: {e}')
        return None

In [17]:
# Funktion um CSV-Dateien zu lesen (Keine Quelle. Excel-Funtkion kopiert)

def read_csv(file_path):
    try:
        df = pd.read_csv(file_path)                     # .read_csv liest CSV-Dateien
        return df.to_string()                           # Wandelt Inhalt in einen String um
    except Exception as e:
        print(f'Fehler beim Lesen der CSV-Datei: {e}')  
        return None


In [18]:
# Funktion um Text-Dateien zu lesen (https://guides.library.upenn.edu/penntdm/python/import_files#:~:text=In%20Python%2C%20you%20can%20use,and%20the%20file%20access%20mode.)
################################### (https://www.python-lernen.de/dateien-auslesen.htm)

def read_txt(file_path):                                        # Die Funtkion funktioniert ohne Imports 
    try:
        with open(file_path, 'r') as file:                      # Später testen ob 'r' oder 'r+' benötigt wird
            text = file.read()                                  # Liest den gesamten Text aus der Datei
        return text
    except Exception as e:
        print(f'Fehler beim Lesen der Textdatei: {e}')          # Fehlermeldung bei einem Fehler
        return None


In [19]:
#Funktion um den text eines PDF zu extraieren   

def read_pdf(file_path):
    """
    Öffnet eine PDF-Datei, extrahiert den Text und gibt ihn zurück.

    Funktionalität:
    - Wählt eine PDF-Datei mit einem Dateidialog aus.
    - Extrahiert den gesamten Text aus der PDF-Datei.
    - Gibt den extrahierten Text zurück oder zeigt eine Fehlermeldung, falls ein Fehler auftritt.
    """
    try:
        # Öffne den Dateidialog, um eine PDF-Datei auszuwählen
        if file_path:
            # Öffne die PDF-Datei
            with open(file_path, "rb") as file:
                reader = PyPDF2.PdfReader(file)
                text = ""

                # Iteriere über alle Seiten der PDF und extrahiere den Text
                for page_num in range(len(reader.pages)):
                    page = reader.pages[page_num]
                    text += page.extract_text()

                return text
        else:
            print("Keine Datei ausgewählt.")
            return None

    except Exception as e:
        print(f"Fehler beim Extrahieren des Textes aus der PDF: {e}")
        return None


In [20]:
# https://pyimagesearch.com/2021/08/23/your-first-ocr-project-with-tesseract-and-python/
def read_image(file_path):
    """
    Öffnet eine Bilddatei und extrahiert den Text mit OCR.

    Funktionalität:
    - Wählt eine Bilddatei (.jpeg, .png, etc.) mit einem Dateidialog aus.                           
    - Verwendet Tesseract OCR, um den Text im Bild zu extrahieren.
    - Gibt den extrahierten Text zurück oder zeigt eine Fehlermeldung, falls ein Fehler auftritt.
    """
    try:
        # Öffne den Dateidialog, um eine Bilddatei auszuwählen

        if file_path:
            # Öffne das Bild mit Pillow
            img = Image.open(file_path)

            # Verwende Tesseract OCR, um den Text zu extrahieren
            text = pytesseract.image_to_string(img)

            return text
        else:
            print("Keine Datei ausgewählt.")
            return None

    except Exception as e:
        print(f"Fehler beim Extrahieren des Textes aus dem Bild: {e}")
        return None

Wir brauchen drei Funktionen :
1. Open_file -> Befehlt um den file_path zu generieren um auf der Datei zu greifen
2. Read_file -> Befehlt um auf die Datei zu greifen und extrahieren.
3. Save_txt -> die extrahierte Datei in einen .txt zu speichern

Funktion die die Datei erkennt, und auf der richtige Funktion zugreift, um die Datei zu oeffnen.

"Wir definiren, welche Tueren mit welchen Schlueßel geoffnet werden konnen."

In [ ]:
def open_file():
    # Öffnet ein Dateiauswahl-Dialogfeld und gibt den Pfad der ausgewählten Datei zurück
    file_types = [
            ("Alle unterstützten Dateien", "*.xlsx;*.docx;*.txt;*.csv;*.pdf;*.jpg;*.jpeg;*.png;*.gif"),
            ("Excel-Dateien", "*.xlsx"),
            ("Word-Dokumente", "*.docx"),
            ("Text-Dateien", "*.txt"),
            ("CSV-Dateien", "*.csv"),
            ("PDF-Dateien", "*.pdf"),
            ("Bild-Dateien", "*.jpg;*.jpeg;*.png;*.gif")
        ]
    file_path = filedialog.askopenfilename(filetypes=file_types)

    if file_path:                                       # wenn wir im folderfinder unsere Datei ausgewaehlt haben:
        print(f"Offnung des Datei-pfads: {file_path}")  # Zeigt der Terminal den ausgewählten Dateipfad an
        text = read_file(file_path)                     # ruft die Funktion read_file() die unten definiert ist
                                                        # read_file() extraiert den text der Datei in eine Variabel 'text'

        if text:        
            save_to_text(text)                          # ruft die Funktion save_to_text() die wir unten definiert haben,
                                                        # sobald die Variabel text definiert wird.
    else:
        print("Offnung der Datei.")                # Damit der Programm nicht spinnt haben wir hier ein text,
    return file_path

In [22]:
# Funktion um Dateiformat zu lesen

def read_file(file_path):               # Wir koennen jetzt die Datei offnen. Nun wollen wir den Inhalt lesen und den Text extraieren.                    
    if not file_path:                   # Wieder eine Fehlermeldungoption falls der Programm spinnt, dann wissen wir wo.
        print('Keine Datei gefunden')   
        return None
    
    file_selection = os.path.splitext(file_path)[1].lower()     # Trennt die Dateiendung vom Dateinamen
    

    readers = {                 # Erstellung eines dict fuer eine bessere Lesbarkeit,
    '.xlsx': read_excel,        # die die verschiedenen Funktionen die wir hoher haben mit
    '.docx': read_word,         # den richtigen Dateitypen linked. Geil oder?
    '.txt': read_txt,
    '.csv': read_csv,
    '.pdf': read_pdf,
    '.jpg': read_image,
    '.jpeg': read_image,
    '.png': read_image,
    '.gif': read_image

    }
    if file_selection in readers:                       # Wir legen die Beschraenkung, die die Funktionen ausfuehrt
        print(f" Zugriff auf der read Funktion : {readers}")       # Wir geben hier  den Dateipfad aus
        text = readers[file_selection](file_path)       # Wir setzen eine Variabel text = machen ein Slicing unsere readers (Funktionen) mit unserer file_selection
                                                        # Wit fuehren also nur die Funktion die die alsgefilterte Dateiendung hat
                                                        
        if text:                                       
            print(f"Extraktes Text: {text[:200]}...")  # Zeigt die 200 Zeichen
        return text
    
    else:
        print('Dateiformat wird nicht unterstützt.')    # wieder ein Fehlermedlung damit wir besser rausfinden koennen,                                                  
        return None                                     # wieso der Programm nicht ausgefuehrt werden kann

In [23]:
# Funktion um den extrahierten Text zu speichern

def save_to_text(text, output_file= 'extracted.txt'):                   # Neue Funktion die ein 'extracted.txt' erstellen muss
    try:
        with open(output_file, 'w', encoding="utf-8") as file:          # wir oeffnen eine Datei mit der Funktion write ('w'), akzeptieren alle Sonderzeichen mit "utf-8" 
            file.write(text)                                            # wir pasten den text der Variabel text in diesem file
            print(f"Text erfolgreich gespeichert in {output_file}")     # Wir geben noch ein bestaetigungsnachricht aus.
    except Exception as e:
        print(f"Fehler beim Speichern der Datei: {e}")                  # Fehlermeldung


In [24]:
  # Button zur Auswahl einer Datei
button = tk.Button(root, text="Datei auswählen", command=open_file) #kein Bock meeeeehr
button.pack(pady=20)

# Tkinter-Hauptloop starten
root.mainloop()  


# TO DO :
# codeslinien eingeben um die 

Ausgewählter Datei-Pfad: C:/Users/Admin/Downloads/image (1).png
 Zugriff auf der read Funktion : {'.xlsx': <function read_excel at 0x000001F807B5FB80>, '.docx': <function read_word at 0x000001F826AC0CA0>, '.txt': <function read_txt at 0x000001F826AC05E0>, '.csv': <function read_csv at 0x000001F807BB4040>, '.pdf': <function read_pdf at 0x000001F826AC0A60>, '.jpg': <function read_image at 0x000001F826AC0E50>, '.jpeg': <function read_image at 0x000001F826AC0E50>, '.png': <function read_image at 0x000001F826AC0E50>, '.gif': <function read_image at 0x000001F826AC0E50>}
Offnung des Datei-pfads


Import file button

https://www.geeksforgeeks.org/create-an-import-file-button-with-tkinter/?ref=header_outind

File explorer :
https://www.geeksforgeeks.org/file-explorer-in-python-using-tkinter/?ref=header_outind

text to speech :
https://www.geeksforgeeks.org/text-to-speech-gui-convertor-using-tkinter-in-python/?ref=header_outind
